<a href="https://colab.research.google.com/github/Victor-Ribeiro-Acosta/Projeto-Math-IBM/blob/main/Biblioteca_popular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import os
import sqlite3
from pandas import DataFrame

In [44]:
# Classe para o usuário
class Usuario:
    def __init__(self, nome, senha):
        self.nome = nome
        self.senha = senha


In [45]:
# Classe para o livro
class Livro:
    def __init__(self, titulo, autor, genero, quantidade):
        self.titulo = titulo
        self.autor = autor
        self.genero = genero
        self.quantidade = quantidade

In [61]:
# Classe para a biblioteca
class Biblioteca:
    def __init__(self):
      self.usuarios_ref = self.lista_usuarios()
      self.livros = self.consultar_catalogo()

    def cadastrar_usuario(self, usuario):
        with sqlite3.connect('Banco_dados_biblioteca.db') as bd:
          cursor = bd.cursor()
          cursor.execute(f'''INSERT INTO usuarios (nome, senha) VALUES(?, ?)''',(usuario.nome, usuario.senha))
          bd.commit()

        print("Usuário cadastrado com sucesso!")
        return True

    def lista_usuarios(self):
        with sqlite3.connect('Banco_dados_biblioteca.db') as banco_dados:
          cursor = banco_dados.cursor()
          dados = cursor.execute('''SELECT * FROM usuarios''').fetchall()
          usuarios = DataFrame(dados, columns=['id','nome','senha'])

        return usuarios

    def cadastrar_livro(self, livro):
        with sqlite3.connect('Banco_dados_biblioteca.db') as bd:
          cursor = bd.cursor()
          cursor.execute(f'''INSERT INTO tabela_livros(Titulo, Autor, Genero, Quantidade) VALUES(?, ?, ?, ?)''',(livro.titulo, livro.autor, livro.genero, livro.quantidade))
          bd.commit()

        print("Livro cadastrado com sucesso!")

    def consultar_catalogo(self):
      with sqlite3.connect('Banco_dados_biblioteca.db') as banco_dados:
        cursor = banco_dados.cursor()
        obras_disponiveis = cursor.execute('SELECT * FROM tabela_livros').fetchall()
        dados = DataFrame(obras_disponiveis, columns=['id','Titulo', 'Autor', 'Genero', 'Quantidade'])
      return dados


In [62]:
# Dicionário para armazenar usuários e livros

biblioteca = Biblioteca()

# Lista de gêneros
generos = ["romance", "novela", "conto", "ficção científica", "fantasia", "mistério", "suspense", "terror", "ficção histórica", "realismo mágico", "ficção especulativa", "ficção de aventura", "ficção policial", "ficção de espionagem", "biografia", "autobiografia", "memórias", "história", "ensaio", "jornalismo literário", "autoajuda", "guias de viagem", "livros de receitas", "livros de referência", "soneto", "ode", "elegia", "haicai", "épico", "lírico", "tragédia", "comédia", "drama histórico", "melodrama", "farsa", "ficção para jovens adultos", "livros ilustrados", "literatura infantil", "graphic novels", "mangá", "light novel"]


In [48]:
# Função para encontrar o gênero mais parecido na lista de gêneros
def encontrar_genero(genero):
    for g in generos:
        if g.startswith(genero.lower()[:4]):
            return g
    return genero

In [50]:
# Função para cadastrar usuário
def cadastrar_usuario():
    while True:
        nome = input("Cadastre  o seu nome de usuário:")
        if len(nome) > 60:
            print("O nome de usuário deve ter até 60 caracteres. Tente novamente.")

        else:
          senha = input("Agora digite uma  senha com 4 dígitos: ")

          if len(senha) != 4:
            print("A senha deve ter 4 dígitos, tente novamente.")

          else:
            break
    usuario = Usuario(nome, senha)
    return biblioteca.cadastrar_usuario(usuario)

In [73]:
# Função para login do usuário
def login():
    nome = input("Digite o seu nome de usuário: ")
    senha = input("Digite a sua senha de 4 dígitos: ")
    if nome in biblioteca.usuarios_ref['nome'].values and senha in biblioteca.usuarios_ref['senha'].values:
      for indice, usuario in biblioteca.usuarios_ref.iterrows():

        if (usuario['nome'] == nome) and (usuario['senha'] == senha):
          print("Login realizado com sucesso!")
          return True

        else:
          print('Usuário ou senha inválidos!')
          return False

    else:
        print("Login falhou.")
        return False

In [52]:
# Função para cadastrar livro
def cadastrar_livro():
    while True:
        titulo = input("Digite o título do livro que quer cadastrar: ")
        if len(titulo) <= 60:
            break
        else:
            print("O título do livro deve ter até 60 caracteres. Tente novamente.")
    while True:
        autor = input("Agora cadastre o nome do autor: ")
        if len(autor) <= 60:
            break
        else:
            print("O nome do autor deve ter até 60 caracteres. Tente novamente.")
    while True:
        genero = input("Digite o gênero do livro: ")
        if len(genero) <= 10 and genero.isalpha():
            genero = encontrar_genero(genero)
            break
        else:
            print("Gênero inválido. Por favor, insira um gênero válido sem números e com até 10 caracteres.")
    while True:
        try:
            quantidade = int(input("Digite a quantidade de exemplares do título que está cadsatrando: "))
            if 0 <= quantidade <= 9999:
                break
            else:
                print("Quantidade inválida. Por favor, insira um número inteiro de 1 a 9999.")
        except ValueError:
            print("Quantidade inválida. Por favor, insira um número inteiro.")
    livro = Livro(titulo, autor, genero, quantidade)
    biblioteca.cadastrar_livro(livro)


In [38]:
with sqlite3.connect('Banco_dados_biblioteca.db') as banco_dados:
    cursor = banco_dados.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS usuarios (id INTEGER PRIMARY KEY AUTOINCREMENT, nome TEXT NOT NULL, senha TEXT NOT NULL)''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS tabela_livros (id INTEGER PRIMARY KEY AUTOINCREMENT, Titulo TEXT NOT NULL, Autor TEXT NOT NULL, Genero TEXT NOT NULL, Quantidade INTEGER)''')


In [85]:
# Função para consultar livro
def consultar_livro():
    opcao = input("Deseja consultar por (1)Título, (2)Autor, (3)Gênero ou (4)Ver todo o cadastro? ")
    if opcao == '1':
        titulo = input("Digite o título do livro: ")
        print(biblioteca.livros[biblioteca.livros['Titulo'] == titulo])


    elif opcao == '2':
        autor = input("Digite o nome do autor: ")
        print(biblioteca.livros[biblioteca.livros['Autor'] == autor])


    elif opcao == '3':
        genero = input("Digite o gênero do livro: ")
        genero = encontrar_genero(genero)
        print(biblioteca.livros[biblioteca.livros['Genero'] == genero])

    elif opcao == '4':
        print(biblioteca.livros)

,id,Titulo,Autor,Genero,Quantidade
0,1,Coraline,Neil Gaiman,terror,5


In [86]:

while True:
  print("Bem-vindo ao sistema de biblioteca!")
  print("1. Cadastrar usuário")
  print("2. Login")
  print("3. Descadastrar usuário")
  print("4. Sair")
  opcao = input("Escolha uma opção: ")
  if opcao == '1':
    if cadastrar_usuario():
      while True:
        print("1. Cadastrar livro")
        print("2. Consultar livro")
        print("3. Descadastrar livro")
        print("4. Sair")
        opcao = input("Escolha uma opção: ")
        if opcao == '1':
          cadastrar_livro()
        elif opcao == '2':
          consultar_livro()
        elif opcao == '3':
          titulo = input("Digite o título do livro que deseja descadastrar: ")
          biblioteca.descadastrar_livro(titulo)
        elif opcao == '4':
          break
  elif opcao == '2':
    if login():
      while True:
        print("1. Cadastrar livro")
        print("2. Consultar livro")
        print("3. Descadastrar livro")
        print("4. Sair")
        opcao = input("Escolha uma opção: ")
        if opcao == '1':
          cadastrar_livro()
        elif opcao == '2':
          consultar_livro()
        elif opcao == '3':
            titulo = input("Digite o título do livro que deseja descadastrar: ")
            biblioteca.descadastrar_livro(titulo)
        elif opcao == '4':
            break
        else:
            print("Login falhou.")
  elif opcao == '3':
    nome = input("Digite o nome do usuário que deseja descadastrar: ")
    biblioteca.descadastrar_usuario(nome)
  elif opcao == '4':
    break

Bem-vindo ao sistema de biblioteca!
1. Cadastrar usuário
2. Login
3. Descadastrar usuário
4. Sair
Escolha uma opção: 2
Digite o seu nome de usuário: Victor Acosta
Digite a sua senha de 4 dígitos: 1234
Login realizado com sucesso!
1. Cadastrar livro
2. Consultar livro
3. Descadastrar livro
4. Sair
Escolha uma opção: 2
Deseja consultar por (1)Título, (2)Autor, (3)Gênero ou (4)Ver todo o cadastro? 1
Digite o título do livro: Coraline
   id    Titulo        Autor  Genero  Quantidade
0   1  Coraline  Neil Gaiman  terror           5
1. Cadastrar livro
2. Consultar livro
3. Descadastrar livro
4. Sair
Escolha uma opção: 4
Bem-vindo ao sistema de biblioteca!
1. Cadastrar usuário
2. Login
3. Descadastrar usuário
4. Sair
Escolha uma opção: 4
